In [30]:

from ngsolve import *
from ngsolve.webgui import Draw
from netgen.csg import *
import scipy.sparse as sp
import numpy as np

In [31]:
# Relevant parameters to play around with

order = 2 # mesh order
max_h = 1 # initial mesh sizesd
C_w = 90 # penalty term weight
refinements = 0 # mumber of refinements of initial mesh

In [32]:
# functions for differential operators on manufactured solutions 

coords = [x,y,z]

def JacobianOfCF(cf):
    """ Function to compute the Jacobi Matrix of a vector coefficient function cf """

    Jac_u_3D = CF((
    cf[0].Diff(x), cf[0].Diff(y), cf[0].Diff(z),
    cf[1].Diff(x), cf[1].Diff(y), cf[1].Diff(z),
    cf[2].Diff(x), cf[2].Diff(y), cf[2].Diff(z)
    ), dims=(3, 3))

    return Jac_u_3D

def GGrad(cf):
    """ Function to compute the gradient of a scalar Coefficient Function """
    gg = [cf.Diff(coords[i]) for i in range(mesh.dim)]
    return CF(tuple(gg))


def GCurl(cf):
    """ Function to compute the curl or rot of vec cf using Jacobian """

    if cf.dim == 1: # if the functions is getting handed a scalar field, its to calculate the curl of the rot..
        curl_rot_u = CF((cf.Diff(y), - cf.Diff(x)))
        return curl_rot_u

    elif mesh.dim == 2:
        rot_u = CF(cf[1].Diff(x) - cf[0].Diff(y))
        return rot_u
    
    elif mesh.dim == 3:
        Jac_u = JacobianOfCF(cf)
        curl_u = CF((Jac_u[2,1] - Jac_u[1,2],  
                    Jac_u[0,2] - Jac_u[2,0],  
                    Jac_u[1,0] - Jac_u[0,1]))
        return curl_u
    

def GDiv(cf):
    """ Function to compute the divergence of a vector coefficient function """

    gd = [cf[i].Diff(coords[i]) for i in range(cf.dim)]
    return CF(sum(gd))

# Functions to calculate h_max

def edge_length(v1, v2, dim):
    return np.sqrt(sum((v1[i] - v2[i])**2 for i in range(dim)))

def squared_distance(v1, v2):
    v1 = np.array(v1)
    v2 = np.array(v2)
    return np.sum((v1 - v2) ** 2)

def cayley_menger_matrix(vertices):
    if len(vertices) != 4:
        raise ValueError("This method is for a tetrahedron, which requires exactly 4 vertices.")

    # Create the Cayley-Menger matrix (5x5)
    C = np.ones((5, 5))
    for i in range(5):
        C[i, i] = 0 

    for i in range(1, 5):
        for j in range(i+1, 5):
            C[i, j] = C[j, i] = squared_distance(vertices[i-1], vertices[j-1])

    return C

def triangle_area(a, b, c):
    s = (a + b + c) / 2  
    return np.sqrt(s * (s - a) * (s - b) * (s - c))

def circumradius_2D(a, b, c):
    area = triangle_area(a, b, c)
    return a * b * c / (4 * area)

def circumradius_3D(vertices):
    C = cayley_menger_matrix(vertices)

    try:
        C_inv = np.linalg.inv(C)
    except np.linalg.LinAlgError:
        raise ValueError("Cayley-Menger matrix is singular or not invertible.")

    M = -2 * C_inv
    circumradius = 0.5 * np.sqrt(M[0, 0])

    return circumradius

def calc_hmax(mesh):
    max_h = 0 
    if mesh.dim == 2:
        for el in mesh.Elements():
            vertices = [mesh[v].point for v in el.vertices]
            a = edge_length(vertices[0], vertices[1], 2)
            b = edge_length(vertices[1], vertices[2], 2)
            c = edge_length(vertices[2], vertices[0], 2)
            circumradius = circumradius_2D(a, b, c)
            max_h = max(max_h, circumradius)
    
    elif mesh.dim == 3:
        for el in mesh.Elements():
            vertices = [mesh[v].point for v in el.vertices]
            circumradius = circumradius_3D(vertices)
            max_h = max(max_h, circumradius)
    
    return max_h

In [33]:
# Create the hollow sphere geometry

r_inner = 1
r_outer = 5 * r_inner

outer_sphere = Sphere(Pnt(0,0,0), r_outer)
outer_sphere.bc("gamma_out")
outer_sphere.bc("gamma_out").maxh(1)

inner_sphere = Sphere(Pnt(0,0,0), r_inner)
inner_sphere.bc("gamma_in")
inner_sphere.bc("gamma_in").maxh(1)


geom = CSGeometry()
geom.Add(outer_sphere - inner_sphere)

mesh = Mesh(geom.GenerateMesh(maxh=max_h))
mesh.Curve(2)

# print(mesh.GetBoundaries())

for i in range(refinements):
    mesh.Refine()

maxh = calc_hmax(mesh)

print("Mesh max_h of hollow sphere is", maxh, "after", refinements, "refinements.")

Mesh max_h of hollow sphere is 1.3041602727193469 after 0 refinements.


In [34]:
print(mesh.GetBoundaries())
# Draw(mesh)

('gamma_out', 'gamma_in')


In [35]:
# Define parameters
mu_0 = 4 * np.pi * 1e-7  # Permeability of free space
m = 1.0                    # Magnitude of the dipole moment
r = 1.0                    # Radius of the sphere

# Define the vector field for the magnetic dipole moment
m_vec = CF((0, 0, m))
r_vec = CF((x, y, z))
r_norm = Norm(r_vec)
r_hat = r_vec / r_norm

# Dot product m.r_hat and resulting field H
m_dot_r_hat = CF(m_vec* r_hat)
H = (1/(4*np.pi*mu_0*r**3)) * (3*m_dot_r_hat*r_hat - m_vec)

In [36]:
V_k = HCurl(mesh, order=order, type1=False)
V_km1 = H1(mesh, order=order+1)

prodSpc = V_k * V_km1

(omega, sigma), (eta, tau) = prodSpc.TnT()

In [37]:
a = BilinearForm(prodSpc)

a += sigma * tau * dx
a += - omega * grad(tau) * dx

a +=  grad(sigma) * eta * dx
a +=  curl(omega) * curl(eta) * dx

Lets unpack the first boundary term from (2c):

$$
+ \, \int_{\Gamma} \text{tr}(\star d\omega_h) \wedge \text{tr}(\eta_h) \tag{2c.1}
$$

If we consider the k-forms in 3D to be: 

$$
\omega , \eta \in \Lambda^1_h(\Omega) \, \Longleftrightarrow \, \omega, \eta \in H(\text{curl}, \Omega)
$$

We can plug in the according vector calculus operations and transform them with help from some identities:

$$
\begin{aligned}
&= \int_{\Gamma} (\text{n} \times \nabla \times \omega_h) \cdot \eta_h \\[10pt]

&= \int_{\Gamma} (\nabla \times \omega_h) \cdot (\eta_h \times \text{n})
\end{aligned}
$$



In [38]:
n = specialcf.normal(mesh.dim)

a += curl(omega) * Cross(eta, n) * ds(skeleton=True, definedon=mesh.Boundaries(".*"))


And the second boundary term from (2c):

$$
\begin{aligned}
&+ \int_{\Gamma} \text{tr} ( \omega_h ) \wedge \text{tr} ( \star d\eta_h ) \tag{2c.2} \\[10pt]

&=\int_{\Gamma} (\omega_h \times \text{n}) \cdot (\nabla \times \eta_h)
\end{aligned}
$$

In [39]:
a += Cross(omega, n) * curl(eta) * ds(skeleton=True, definedon=mesh.Boundaries(".*"))

And the pentalty Nitsche term:

$$
+ \, \frac{C_w}{h} \langle \text{tr}\, \omega, \text{tr}\, \eta \rangle_{L^2(\Gamma)} \tag{2d.1}
$$

In [40]:
h = specialcf.mesh_size
a += (C_w / h) * Cross(omega, n) * Cross(eta, n) * ds(skeleton=True, definedon=mesh.Boundaries(".*"))

In [41]:
A = 0.1  # Amplitude of the pulse
sigma_pulse = 0.2  # Width of the Gaussian pulse
r0 = (0.5, 0.5, 0.5)  # Center of the Gaussian pulse
n_pulse = (0, 0, 1)  # Direction of the pulse (unit vector)

gauss_pulse = CF((
    A * exp(-((x - r0[0])**2 + (y - r0[1])**2 + (z - r0[2])**2) / (2 * sigma_pulse**2)) * n_pulse[0],
    A * exp(-((x - r0[0])**2 + (y - r0[1])**2 + (z - r0[2])**2) / (2 * sigma_pulse**2)) * n_pulse[1],
    A * exp(-((x - r0[0])**2 + (y - r0[1])**2 + (z - r0[2])**2) / (2 * sigma_pulse**2)) * n_pulse[2]
))

omega_m_sinusoids = CF((sin(pi*x)*sin(pi*y)*sin(pi*z), 
              sin(pi*x)*sin(pi*y)*sin(pi*z), 
              sin(pi*x)*sin(pi*y)*sin(pi*z)))



omega_m_zero = CF((0,0,0))

omega_m = omega_m_zero

hL_omega = GCurl(GCurl(omega_m)) - GGrad(GDiv(omega_m))

omega_gamma = omega_m # this is used to compute the tangential values on the boundary


f = LinearForm(prodSpc)
f += hL_omega * eta * dx
f += (C_w / h) * Cross(n, Cross(H, n)) * Cross(n, Cross(eta, n)) * ds(skeleton=True, definedon="gamma_in")
f += Cross(n, Cross(H, n)) * Cross(n, curl(eta)) * ds(skeleton=True, definedon="gamma_in")

In [42]:
a.Assemble()
f.Assemble()

rows,cols,vals = a.mat.COO()
A = sp.csr_matrix((vals,(rows,cols)))
#cond_nr = np.linalg.cond(A.todense())

sol = GridFunction(prodSpc)
res = f.vec-a.mat * sol.vec
inv = a.mat.Inverse(freedofs=prodSpc.FreeDofs(), inverse="pardiso")
sol.vec.data += inv * res

gf_omega , gf_sigma = sol.components

curl_omega = curl(gf_omega)
grad_sigma = grad(gf_sigma)

# print("Matrix dimensions:", a.mat.height, "x", a.mat.width)
# #print("Matrix Condition Number: ", cond_nr)
# print("Residual: ", Norm(res))
# print("L2 Error omega: ", sqrt(Integrate((gf_omega - omega_m)**2, mesh)))
# print("L2 Error curl(omega): ", sqrt(Integrate((curl(gf_omega) - GCurl(omega_m))**2, mesh)))
# print("L2 Error sigma:: ", sqrt(Integrate((gf_sigma + GDiv(omega_m))**2, mesh)))
# print("L2 Error grad(sigma): ", sqrt(Integrate((grad(gf_sigma) + GGrad(GDiv(omega_m)))**2, mesh)))



In [45]:
from ngsolve.internal import viewoptions, visoptions
viewoptions.drawfilledtrigs = 0
Draw(curl(gf_omega), mesh)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

BaseWebGuiScene